In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [16]:
records = pd.read_excel("coffee_shop_sales.xlsx", index_col="transaction_id", nrows=100)
records.head()

,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
transaction_id,,,,,,,,,,
1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg
2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm
5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg


In [20]:
records.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 1 to 100
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_date  100 non-null    datetime64[ns]
 1   transaction_time  100 non-null    object        
 2   transaction_qty   100 non-null    int64         
 3   store_id          100 non-null    int64         
 4   store_location    100 non-null    object        
 5   product_id        100 non-null    int64         
 6   unit_price        100 non-null    float64       
 7   product_category  100 non-null    object        
 8   product_type      100 non-null    object        
 9   product_detail    100 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 8.6+ KB


In [58]:
transaction_datetime = pd.to_datetime(records["transaction_date"].astype(str) + " " + records["transaction_time"].astype(str))
transaction_datetime = pd.Series(transaction_datetime, name="transaction_datetime")

In [60]:
transaction_datetime

transaction_id
1     2023-01-01 07:06:11
2     2023-01-01 07:08:56
3     2023-01-01 07:14:04
4     2023-01-01 07:20:24
5     2023-01-01 07:22:41
              ...        
96    2023-01-01 10:35:47
97    2023-01-01 10:41:53
98    2023-01-01 10:47:53
99    2023-01-01 10:49:13
100   2023-01-01 10:49:51
Name: transaction_datetime, Length: 100, dtype: datetime64[ns]